In [1]:
!nvidia-smi

Sun Mar  7 02:37:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    17W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Colab如何加装Google drive 硬盘：https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/'Shared drives'/JXNU_C/HXM/IPC_to_CLC/

/content/drive/Shared drives/JXNU_C/HXM/IPC_to_CLC


In [4]:
!pwd

/content/drive/Shared drives/JXNU_C/HXM/IPC_to_CLC


In [5]:
!ls

Bi_LSTM.ipynb
data_KFold_1
drive-download-20210306T124752Z-001.zip
error_log
ipc_to_clc_data_and_result_类目相似度_结果备份
log
other_network
output
sentence_transformers
si_model.ipynb
test_sentence_transformers.ipynb
Text_CNN.ipynb
train_sentence_transformers.ipynb
Transformer_Encoder.ipynb


In [6]:
!pip install ipdb transformers sentence-transformers

## 基于sentence_transformers实现预训练语言模型孪生网络，并进行训练。

In [7]:
import os
import math
import logging
import ipdb
import torch
import json
import sys


from torch import nn
from torch.utils.data import DataLoader
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, InputExample, LoggingHandler, SentenceTransformer, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
from datetime import datetime
from tqdm import tqdm
from typing import List


#设置随机种子，保证实验的可重复性
seed = 42

torch.manual_seed(seed)  # 为CPU设置随机种子
torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
#如果使用多个GPU，应该使用torch.cuda.manual_seed_all()为所有的GPU设置种子

#用以保证模型的可重复性，实验的可重复性
torch.backends.cudnn.deterministic = True

sys.path.append('/content/drive/Shared drives/JXNU_C/HXM/IPC_to_CLC')

In [8]:
#设置日志格式
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])


#模型名字  distiluse-base-multilingual-cased-v2 
model_name = "bert-base-chinese"
#是否自定义模型(与模型名字有关)
custom_model = True

batch_size = 64
num_epochs = 24
weight_decay = 0.01

# 损失函数类型： sample_ContrastiveLoss   sample_MultipleNegativesRankingLoss   sample_TripletLoss
loss_type = "sample_ContrastiveLoss"

# 五折交叉验证，第几折选择: 0 1 2 3 4
kfold = "4"

#训练好的模型保存路径
model_save_path = "output/" + "LSTM-" + str(kfold) + "-" + str(num_epochs) + "-" + "training_ipc-to-clc_" + model_name + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

#判断GPU是否可用
device = "cuda" if torch.cuda.is_available() else 'cpu'



#自定义模型，包括transformers中的模型也可以用(可以自定义模型框架)
if custom_model:
    # Use model_name for mapping tokens to embeddings
    word_embedding_model = models.Transformer(model_name, model_args={'output_hidden_states': True})

    lstm_model = models.LSTM(word_embedding_model.get_word_embedding_dimension(), 
                           hidden_dim=768, 
                           num_layers=1, 
                           dropout=0, 
                           bidirectional=True)

    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(lstm_model.get_word_embedding_dimension(),
                                  pooling_mode_mean_tokens=True,
                                  pooling_mode_cls_token=False,
                                  pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, lstm_model, pooling_model], device=device)

#加载sentence_transformers已经训练过的模型
else:
    model = SentenceTransformer(model_name, device=device)


#train文件路径
train_dataset_path = "data_KFold_1/" + loss_type + "/data_sample_" + kfold + "/ipc_clc_train.txt"

#valid文件路径(这个只有 sample_MultipleNegativesRankingLoss 需要，因为训练数据中没有label，需要在训练前将验证数据从训练集中分离出来)
valid_dataset_path = "data_KFold_1/" + loss_type + "/data_sample_" + kfold + "/ipc_clc_valid.txt"


#读入训练数据
logging.info("读入训练数据...")

train_shuliang = 0

train_samples = []
if loss_type == "sample_MultipleNegativesRankingLoss":
    with open(train_dataset_path, 'r', encoding='utf8') as fIn:
    for row in fIn:
        train_shuliang += 1
        row = row.strip()
        row = row.split('\t')
        train_samples.append(InputExample(texts=[row[0], row[1]])) # MultipleNegativesRankingLoss 无label
else:
    with open(train_dataset_path, 'r', encoding='utf8') as fIn:
        train_valid_number = 0
        for row in fIn:
            train_valid_number = train_valid_number + 1
            if train_valid_number % 5 == 0:
                continue
        else:
            train_shuliang += 1
            row = row.strip()
            row = row.split('\t')
            if loss_type == "sample_ContrastiveLoss":
                train_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) # ContrastiveLoss 有label
            else:
                train_samples.append(InputExample(texts=[row[0], row[1], row[2]])) # TripletLoss 无label

print('训练集大小：' + str(train_shuliang))        

#构造训练数据 DataLoader
train_data = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)


#定义训练的损失函数
if loss_type == "sample_ContrastiveLoss":
    train_loss = losses.OnlineContrastiveLoss(model=model, margin=0.5)  #有label
elif loss_type == "sample_MultipleNegativesRankingLoss":
    train_loss = losses.MultipleNegativesRankingLoss(model=model)  #无label
else:
    train_loss = losses.TripletLoss(model=model, triplet_margin=1)  #无label 默认triplet_margin=5




#读入验证集数据
logging.info("读入验证集数据...")

valid_shuliang = 0

valid_samples = []
#sample_MultipleNegativesRankingLoss直接从文件中读取，其他loss的验证集将直接从训练数据中得到，1/5的训练集数据将作为验证集
#需要注意的是，如果只有一种label，则evaluator无法进行验证，所以是二分类的要保证验证集中两种类别都有
if loss_type == "sample_MultipleNegativesRankingLoss":
    with open(valid_dataset_path, 'r', encoding='utf8') as fIn:
        for row in fIn:
            valid_shuliang += 1
            row = row.strip()
            row = row.split('\t')
            valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) #验证集需要 label
else:   
    with open(train_dataset_path, 'r', encoding='utf8') as fIn:
        train_valid_number = 0
        for row in fIn:
            train_valid_number = train_valid_number + 1
            if train_valid_number % 5 == 0:
                valid_shuliang += 1
                row = row.strip()
                row = row.split('\t')
                if loss_type == "sample_ContrastiveLoss":
                    valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) #验证集需要 label
                else:
                    valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(1))) # TripletLoss pos
                    valid_samples.append(InputExample(texts=[row[0], row[2]], label=int(0))) # TripletLoss neg
            else:
                continue

print('验证集大小：' + str(valid_shuliang))  

#构造验证集数据 DataLoader
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_samples, name='ipc-clc-valid')


#warmup步数  这个也有很大的随机性
warmup_steps = math.ceil(len(train_samples) * num_epochs / batch_size * 0.15) #15% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


#模型训练
model.fit(train_objectives = [(train_dataloader, train_loss)],
          epochs = num_epochs,
          #optimizer_class = 'transformers.optimization.AdamW',
          optimizer_params = {'correct_bias': False, 'eps': 1e-06, 'lr': 2e-05},
          scheduler = 'WarmupLinear',
          evaluator = evaluator,
          evaluation_steps = 10,
          save_best_model = True,
          weight_decay = weight_decay,
          warmup_steps = warmup_steps,
          max_grad_norm = 1,
          output_path = model_save_path
         )

2021-03-07 02:37:07 - 读入训练数据...
训练集大小：1280
2021-03-07 02:37:07 - 读入验证集数据...
验证集大小：320
2021-03-07 02:37:07 - Warmup-steps: 72


2021-03-07 02:37:21 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 0 after 10 steps:
2021-03-07 02:37:22 - Cosine-Similarity :	Pearson: 0.9063	Spearman: 0.8514
2021-03-07 02:37:22 - Manhattan-Distance:	Pearson: 0.8962	Spearman: 0.8523
2021-03-07 02:37:22 - Euclidean-Distance:	Pearson: 0.8955	Spearman: 0.8522
2021-03-07 02:37:22 - Dot-Product-Similarity:	Pearson: 0.8783	Spearman: 0.8448
2021-03-07 02:37:22 - Save model to output/LSTM-4-24-training_ipc-to-clc_bert-base-chinese-2021-03-07_02-37-05
2021-03-07 02:37:35 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 0 after 20 steps:
2021-03-07 02:37:37 - Cosine-Similarity :	Pearson: 0.9177	Spearman: 0.8563
2021-03-07 02:37:37 - Manhattan-Distance:	Pearson: 0.9190	Spearman: 0.8572
2021-03-07 02:37:37 - Euclidean-Distance:	Pearson: 0.9180	Spearman: 0.8573
2021-03-07 02:37:37 - Dot-Product-Similarity:	Pearson: 0.8659	Spearman: 0.8375
2021-03-07 02:37:37 - Save mode

2021-03-07 02:37:52 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 1 after 10 steps:
2021-03-07 02:37:53 - Cosine-Similarity :	Pearson: 0.9261	Spearman: 0.8598
2021-03-07 02:37:53 - Manhattan-Distance:	Pearson: 0.9275	Spearman: 0.8597
2021-03-07 02:37:53 - Euclidean-Distance:	Pearson: 0.9263	Spearman: 0.8599
2021-03-07 02:37:53 - Dot-Product-Similarity:	Pearson: 0.8825	Spearman: 0.8499
2021-03-07 02:37:53 - Save model to output/LSTM-4-24-training_ipc-to-clc_bert-base-chinese-2021-03-07_02-37-05
2021-03-07 02:38:06 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 1 after 20 steps:
2021-03-07 02:38:07 - Cosine-Similarity :	Pearson: 0.9197	Spearman: 0.8600
2021-03-07 02:38:07 - Manhattan-Distance:	Pearson: 0.9237	Spearman: 0.8597
2021-03-07 02:38:07 - Euclidean-Distance:	Pearson: 0.9224	Spearman: 0.8599
2021-03-07 02:38:07 - Dot-Product-Similarity:	Pearson: 0.8642	Spearman: 0.8477
2021-03-07 02:38:07 - Save mode

2021-03-07 02:38:22 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 2 after 10 steps:
2021-03-07 02:38:24 - Cosine-Similarity :	Pearson: 0.9141	Spearman: 0.8597
2021-03-07 02:38:24 - Manhattan-Distance:	Pearson: 0.9200	Spearman: 0.8594
2021-03-07 02:38:24 - Euclidean-Distance:	Pearson: 0.9187	Spearman: 0.8593
2021-03-07 02:38:24 - Dot-Product-Similarity:	Pearson: 0.8516	Spearman: 0.8437
2021-03-07 02:38:35 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 2 after 20 steps:
2021-03-07 02:38:36 - Cosine-Similarity :	Pearson: 0.9292	Spearman: 0.8572
2021-03-07 02:38:36 - Manhattan-Distance:	Pearson: 0.9280	Spearman: 0.8574
2021-03-07 02:38:36 - Euclidean-Distance:	Pearson: 0.9271	Spearman: 0.8576
2021-03-07 02:38:36 - Dot-Product-Similarity:	Pearson: 0.9054	Spearman: 0.8507

2021-03-07 02:38:36 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 2:
2021-03-07 02:38:38 - Cosine

2021-03-07 02:38:48 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 3 after 10 steps:
2021-03-07 02:38:50 - Cosine-Similarity :	Pearson: 0.9072	Spearman: 0.8555
2021-03-07 02:38:50 - Manhattan-Distance:	Pearson: 0.9170	Spearman: 0.8574
2021-03-07 02:38:50 - Euclidean-Distance:	Pearson: 0.9169	Spearman: 0.8575
2021-03-07 02:38:50 - Dot-Product-Similarity:	Pearson: 0.8764	Spearman: 0.8472
2021-03-07 02:39:01 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 3 after 20 steps:
2021-03-07 02:39:02 - Cosine-Similarity :	Pearson: 0.9060	Spearman: 0.8566
2021-03-07 02:39:02 - Manhattan-Distance:	Pearson: 0.9162	Spearman: 0.8591
2021-03-07 02:39:02 - Euclidean-Distance:	Pearson: 0.9163	Spearman: 0.8594
2021-03-07 02:39:02 - Dot-Product-Similarity:	Pearson: 0.8708	Spearman: 0.8474

2021-03-07 02:39:02 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 3:
2021-03-07 02:39:04 - Cosine

...

2021-03-07 02:46:24 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 19 after 10 steps:
2021-03-07 02:46:25 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8623
2021-03-07 02:46:25 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:46:25 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:46:25 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430
2021-03-07 02:46:36 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 19 after 20 steps:
2021-03-07 02:46:38 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:46:38 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:46:38 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:46:38 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430

2021-03-07 02:46:38 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 19:
2021-03-07 02:46:39 - Cos

2021-03-07 02:46:51 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 20 after 10 steps:
2021-03-07 02:46:52 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:46:52 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:46:52 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:46:52 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430
2021-03-07 02:47:03 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 20 after 20 steps:
2021-03-07 02:47:05 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:47:05 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:47:05 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:47:05 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430

2021-03-07 02:47:05 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 20:
2021-03-07 02:47:07 - Cos

2021-03-07 02:47:18 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 21 after 10 steps:
2021-03-07 02:47:20 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:47:20 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:47:20 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:47:20 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430
2021-03-07 02:47:31 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 21 after 20 steps:
2021-03-07 02:47:32 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:47:32 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:47:32 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:47:32 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430

2021-03-07 02:47:32 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 21:
2021-03-07 02:47:34 - Cos

2021-03-07 02:47:45 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 22 after 10 steps:
2021-03-07 02:47:46 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:47:46 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:47:46 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:47:46 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430
2021-03-07 02:47:58 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 22 after 20 steps:
2021-03-07 02:47:59 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:47:59 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:47:59 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:47:59 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430

2021-03-07 02:47:59 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 22:
2021-03-07 02:48:01 - Cos

2021-03-07 02:48:12 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 23 after 10 steps:
2021-03-07 02:48:14 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:48:14 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:48:14 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:48:14 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430
2021-03-07 02:48:25 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 23 after 20 steps:
2021-03-07 02:48:27 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:48:27 - Manhattan-Distance:	Pearson: 0.9245	Spearman: 0.8629
2021-03-07 02:48:27 - Euclidean-Distance:	Pearson: 0.9241	Spearman: 0.8631
2021-03-07 02:48:27 - Dot-Product-Similarity:	Pearson: 0.8618	Spearman: 0.8430

2021-03-07 02:48:27 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 23:
2021-03-07 02:48:29 - Cos

In [9]:
#利用训练好的模型得到测试集结果
model = SentenceTransformer(model_save_path)

evaluator(model, output_path=model_save_path)

2021-03-07 02:48:29 - Load pretrained SentenceTransformer: output/LSTM-4-24-training_ipc-to-clc_bert-base-chinese-2021-03-07_02-37-05
2021-03-07 02:48:29 - Load SentenceTransformer from folder: output/LSTM-4-24-training_ipc-to-clc_bert-base-chinese-2021-03-07_02-37-05
2021-03-07 02:48:31 - Use pytorch device: cuda
2021-03-07 02:48:31 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset:
2021-03-07 02:48:33 - Cosine-Similarity :	Pearson: 0.9216	Spearman: 0.8622
2021-03-07 02:48:33 - Manhattan-Distance:	Pearson: 0.9244	Spearman: 0.8628
2021-03-07 02:48:33 - Euclidean-Distance:	Pearson: 0.9240	Spearman: 0.8631
2021-03-07 02:48:33 - Dot-Product-Similarity:	Pearson: 0.8619	Spearman: 0.8431


0.8631203141585486

## 利用预训练语言模型进行验证（这里只需要测试数据，k折交叉验证可以直接改变测试数据）

In [10]:
#可选，如果不进行上面的训练，则这里不注释，第0个文件中保存有全部1000条数据
#import torch
#from sentence_transformers import SentenceTransformer, util
#from tqdm import tqdm
#kfold = "0"

#读入IPC和CLC测试数据
read_ipc_file = open("data_KFold_1/" + "origin/" + "data_origin_" + kfold + "/ipc_valid.txt", 'r', encoding='utf-8')
read_clc_file = open("data_KFold_1/" + "origin/" + "data_origin_" + kfold + "/clc_valid.txt", 'r', encoding='utf-8')

#存储IPC和CLC的分别类号(key)和匹配字符串(value)
ipc_key_list = []
ipc_value_list = []

clc_key_list = []
clc_value_list = []

for line in read_ipc_file:
    line = line.strip()
    line = line.split(' ')
    assert len(line) == 2, 'IPC.txt文件中只应该有两列，每一列用一个空格隔开！' + str(line[0])
    ipc_key_list.append(str(line[0]))
    ipc_value_list.append(str(line[1]))
    
        
for line in read_clc_file:
    line = line.strip()
    line = line.split(' ')
    assert len(line) == 2, 'CLC.txt文件中只应该有两列，每一列用一个空格隔开！' + str(line[0])
    clc_key_list.append(str(line[0]))
    clc_value_list.append(str(line[1]))
    
assert len(ipc_key_list) == len(clc_key_list), 'IPC.txt文件和CLC.txt文件要一一对应，行数要一致！'


match_clc_key_dict = dict()

for ipc_key_i, target in enumerate(ipc_key_list):
    match_clc_key_dict[str(ipc_key_i)] = []
    for ipc_key_j, every_ipc_key in enumerate(ipc_key_list):
        if every_ipc_key == target:
            match_clc_key_dict[str(ipc_key_i)].append(clc_key_list[ipc_key_j])

#用于计算IPC每一类别包含的数量
ipc_section_number_dict = {
    'A' : 0,
    'B' : 0,
    'C' : 0,
    'D' : 0,
    'E' : 0,
    'F' : 0,
    'G' : 0,
    'H' : 0
}
for ipc_key_i in ipc_key_list:
    ipc_section_number_dict[str(ipc_key_i[0])] += 1



"""
思路：
利用预训练语言模型对IPC和CLC字符串进行编码，得到句子级别的语境向量，
之后对两者进行语义相似度计算（余弦相似度）。

步骤：
1.我们将IPC作为查询字符串，CLC作为待匹配的字符串。

2.一条IPC字符串需要与每一条CLC字符串进行语义相似度计算（余弦相似度）。

3.计算完毕后，取出相似度最高的前n条CLC分类号，作为与当前IPC形成映射的CLC分类号。

4.计算准确率，TopK准确率。
"""

#判断GPU是否可用
device = "cuda" if torch.cuda.is_available() else 'cpu'
#模型路径 （optional:  model_save_path   "bert-base-chinese"   "distiluse-base-multilingual-cased-v2"  "hfl/chinese-bert-wwm"）
model_path = model_save_path

#加载预训练语言模型（可更换,传入模型名称/模型路径）
model = SentenceTransformer(model_path, device=device)

#将CLC查询字符串转化为语境向量，并转化为tensor
clc_corpus_embeddings = model.encode(clc_value_list, convert_to_tensor=True, device=device)

'''
ipdb> len(clc_value_list)
200

ipdb> clc_corpus_embeddings.shape
torch.Size([200, 768])
'''

#可规定TopK准确率
top_k = 10

#存储相似度最高的前n条CLC分类号
clc_top_k_results = dict()

#存储匹配正确的数量(只取匹配度最高的)
correct = 0

#存储TopK中匹配正确的数量
top_k_correct = 0

#存储top2-TopK中匹配正确的数量(包含了topK)
top2_to_topk_correct_dict = {
    '2' : 0,
    '3' : 0,
    '4' : 0,
    '5' : 0,
    '6' : 0,
    '7' : 0,
    '8' : 0,
    '9' : 0,
    '10' : 0
}

#存储每一个IPC类别的正确个数
ipc_section_correct_dict = {
    'A' : 0,
    'B' : 0,
    'C' : 0,
    'D' : 0,
    'E' : 0,
    'F' : 0,
    'G' : 0,
    'H' : 0
}

#存储相似度最高匹配错误的ipc和clc句对
error_match_ipc_clc_pair = []

#存储topk匹配错误的ipc和clc句对
topk_error_match_ipc_clc_pair = []

pbar = tqdm(ipc_value_list)
for ipc_i, ipc_query in enumerate(pbar):
    
    #将IPC查询字符串转化为语境向量，并转化为tensor
    ipc_query_embedding = model.encode(ipc_query, convert_to_tensor=True, device=device)
    '''
    ipdb> ipc_query
    '鞋类;鞋类的特征;鞋类的部件;鞋帮;靴腿;增强件;其他单个鞋部件;鞋帮;靴腿;'

    ipdb> ipc_query_embedding .shape
    torch.Size([768])
    '''
    #计算两者的语义相似度计算（余弦相似度）
    cos_scores = util.pytorch_cos_sim(ipc_query_embedding, clc_corpus_embeddings)[0]
    '''
    ipdb> ipc_query_embedding.shape
    torch.Size([768])

    ipdb> clc_corpus_embeddings.shape
    torch.Size([200, 768])

    ipdb> cos_scores .shape
    torch.Size([200])
    '''
    cos_scores = cos_scores.cpu()
    

    #使用 torch.topk 找到相似度最高的前n条CLC分类号
    top_results = torch.topk(cos_scores, k=top_k)
    '''
    ipdb> top_results 
    torch.return_types.topk(
    values=tensor([0.9510, 0.8601, 0.8227, 0.8227, 0.8227, 0.7801, 0.7547, 0.7545, 0.7543,
            0.7527]),
    indices=tensor([  0,  11, 118, 126, 119,  36,  47,  23,  90,  74]))
    '''
    clc_top_k_results[str(ipc_i)] = []
    to_match_clc_key_list = []
    
    #只有前K个结果
    for score, idx in zip(top_results[0], top_results[1]):
        clc_top_k_results[str(ipc_i)].append((str(clc_key_list[idx]), str(clc_value_list[idx]), score))
        to_match_clc_key_list.append(str(clc_key_list[idx]))
    
    #匹配正确的数量(只取第一个匹配度最高的)
    if clc_top_k_results[str(ipc_i)][0][0] in match_clc_key_dict[str(ipc_i)]:
        correct = correct + 1
        ipc_section_correct_dict[str(ipc_key_list[ipc_i][0])] += 1
    else:
        error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                        '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                        str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                        str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                        str(clc_top_k_results[str(ipc_i)][0][1]))
    
    #TopK中匹配正确的数量(10个的)
    if len(match_clc_key_dict[str(ipc_i)]) == 1:
        assert match_clc_key_dict[str(ipc_i)][0] == str(clc_key_list[ipc_i]), 'TopK匹配有问题！--1'
        if str(clc_key_list[ipc_i]) in to_match_clc_key_list:
            top_k_correct = top_k_correct + 1
        else:
            topk_error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                          '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                          str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                          str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                          str(clc_top_k_results[str(ipc_i)][0][1]))
    elif len(match_clc_key_dict[str(ipc_i)]) > 1:
        for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
            if str(each_clc_key_1) in to_match_clc_key_list:
                top_k_correct = top_k_correct + 1
                break
            else:
                topk_error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                            '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                            str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                            str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                            str(clc_top_k_results[str(ipc_i)][0][1]))
    else:
        print('TopK匹配有问题！--2')

    
    #计算tok2-topK每一个的准确率
    for topk_index in range(1, top_k):
        for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
            if str(each_clc_key_1) in to_match_clc_key_list[:topk_index+1]:
                top2_to_topk_correct_dict[str(topk_index+1)] += 1
                break

    pbar.set_description("Processing %s" % str(ipc_i+1))
    
    
#计算准确率，TopK准确率
ipc_count_number = len(ipc_key_list)
acc = correct/ipc_count_number
top_k_acc = top_k_correct/ipc_count_number
print('\nFinal acc: {:.2f}% .'.format(acc*100))
print('\nFinal top_{}_acc: {:.2f}% .'.format(top_k, top_k_acc*100))
print('\n--------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
    print('\nFinal top_{}_acc: {:.2f}% .'.format(topk_name, (top_correct_num/ipc_count_number)*100))

2021-03-07 02:48:33 - Load pretrained SentenceTransformer: output/LSTM-4-24-training_ipc-to-clc_bert-base-chinese-2021-03-07_02-37-05
2021-03-07 02:48:33 - Load SentenceTransformer from folder: output/LSTM-4-24-training_ipc-to-clc_bert-base-chinese-2021-03-07_02-37-05


  0%|          | 0/200 [00:00<?, ?it/s]

Processing 1:   0%|          | 0/200 [00:00<?, ?it/s]

Processing 2:   1%|          | 2/200 [00:00<00:16, 12.28it/s]

Processing 3:   1%|          | 2/200 [00:00<00:16, 12.28it/s]

...

Processing 197:  98%|█████████▊| 197/200 [00:21<00:00,  9.10it/s]

Processing 198:  98%|█████████▊| 197/200 [00:21<00:00,  9.10it/s]

Processing 199: 100%|█████████▉| 199/200 [00:21<00:00,  9.58it/s]

Processing 200: 100%|██████████| 200/200 [00:21<00:00,  9.35it/s]



Final acc: 88.00% .

Final top_10_acc: 99.00% .

--------------------


Final top_2_acc: 96.50% .

Final top_3_acc: 97.00% .

Final top_4_acc: 98.00% .

Final top_5_acc: 99.00% .

Final top_6_acc: 99.00% .

Final top_7_acc: 99.00% .

Final top_8_acc: 99.00% .

Final top_9_acc: 99.00% .

Final top_10_acc: 99.00% .


In [11]:
#结果统计
print('总数：{}\ntop1 正确个数：{}\ntop1 错误个数：{}\n'.format(ipc_count_number, correct, len(error_match_ipc_clc_pair)))

print('总数：{}\ntop10 正确个数：{}\ntop10 错误个数：{}\n'.format(ipc_count_number, top_k_correct, len(topk_error_match_ipc_clc_pair)))


print('\n---------------------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
    print('总数：{}\ntop{} 正确个数：{}\ntop{} 错误个数：{}\n'.format(ipc_count_number, topk_name, top_correct_num, topk_name, ipc_count_number-top_correct_num))

总数：200
top1 正确个数：176
top1 错误个数：24

总数：200
top10 正确个数：198
top10 错误个数：2


---------------------------------

总数：200
top2 正确个数：193
top2 错误个数：7

总数：200
top3 正确个数：194
top3 错误个数：6

总数：200
top4 正确个数：196
top4 错误个数：4

总数：200
top5 正确个数：198
top5 错误个数：2

总数：200
top6 正确个数：198
top6 错误个数：2

总数：200
top7 正确个数：198
top7 错误个数：2

总数：200
top8 正确个数：198
top8 错误个数：2

总数：200
top9 正确个数：198
top9 错误个数：2

总数：200
top10 正确个数：198
top10 错误个数：2



In [12]:
#计算每一个IPC类别的正确个数和正确率（只有top1的）
for sec_i, sec_cor in ipc_section_correct_dict.items():
    print('类别：{}，总数为：{}，正确个数为：{}，错误个数为：{}，正确率为：{:.2f}%\n'.format(sec_i, ipc_section_number_dict[sec_i], sec_cor, ipc_section_number_dict[sec_i]-sec_cor, (sec_cor/ipc_section_number_dict[sec_i])*100))

类别：A，总数为：18，正确个数为：17，错误个数为：1，正确率为：94.44%

类别：B，总数为：40，正确个数为：31，错误个数为：9，正确率为：77.50%

类别：C，总数为：28，正确个数为：25，错误个数为：3，正确率为：89.29%

类别：D，总数为：19，正确个数为：17，错误个数为：2，正确率为：89.47%

类别：E，总数为：18，正确个数为：16，错误个数为：2，正确率为：88.89%

类别：F，总数为：27，正确个数为：23，错误个数为：4，正确率为：85.19%

类别：G，总数为：24，正确个数为：23，错误个数为：1，正确率为：95.83%

类别：H，总数为：26，正确个数为：24，错误个数为：2，正确率为：92.31%



In [13]:
#可选，如果不进行上面的训练，则这里不注释
loss_type = 'Bi_LSTM'
#kfold = model_path
#num_epochs = 'no_train'

#相似度最高的错误匹配（topk=1）
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'top1_error_text_log.txt', 'w', encoding='utf-8')

for k_v in error_match_ipc_clc_pair:
    f_write.write(str(k_v) + '\n')

f_write.close()

In [14]:
#topk的错误匹配结果
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'topk_error_text_log.txt', 'w', encoding='utf-8')

for k_v in topk_error_match_ipc_clc_pair:
    f_write.write(str(k_v) + '\n')

f_write.close()

In [15]:
#topk的匹配结果
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'topk_error_score_log.txt', 'w', encoding='utf-8')

for k_index, v_list_tuple in clc_top_k_results.items():
    for each_tuple in v_list_tuple:
        f_write.write(ipc_key_list[int(k_index)] + '----' + each_tuple[0] + each_tuple[1] + str(each_tuple[2].item()) + '\n')

    f_write.write('------------------------------------------------------------------------------' + '\n')

f_write.close()